In [16]:
import pandas as pd
import datetime

In [258]:
df_snow = pd.read_excel("data/Климат Средние величины.xlsx")

In [259]:
df_snow

,id_station,station,1,2,3,4,5,6,7,8,9,10,11,12,start,end
0,24817,Ербогачен,15.0,17.0,31.0,46.0,48.0,31.0,30.0,37.0,46.0,40.0,29.0,19.0,2022-10-11,2022-04-21
1,24918,Преображенка,17.0,21.0,35.0,54.0,52.0,27.0,24.0,34.0,44.0,40.0,30.0,21.0,2022-10-13,2022-04-15
2,29393,Червянка,23.0,25.0,40.0,54.0,49.0,30.0,39.0,47.0,48.0,49.0,45.0,32.0,2022-10-18,2022-03-31
3,29498,Шиткино,26.0,33.0,45.0,54.0,74.0,51.0,59.0,60.0,48.0,51.0,53.0,44.0,2022-10-18,2022-03-24
4,29590,Новочунка,27.0,34.0,44.0,53.0,70.0,38.0,46.0,57.0,48.0,50.0,55.0,44.0,2022-10-19,2022-03-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,30812,Култук,14.0,17.0,39.0,65.0,71.0,77.0,81.0,97.0,59.0,38.0,30.0,22.0,2022-10-27,2022-03-30
65,30815,Хамар-Дабан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
66,30818,Байкальск,12.0,13.0,28.0,47.0,55.0,55.0,47.0,71.0,44.0,29.0,24.0,18.0,2022-10-19,2022-04-10
67,UIBB,Братск Аэропорт,24.0,26.0,31.0,37.0,43.0,30.0,41.0,54.0,37.0,35.0,40.0,32.0,2022-10-18,2022-03-31


In [420]:
def changing_columns():
    cols = list(df_temp.columns) 
    cols.append('temp')
    df_temp.reset_index(inplace = True)
    df_temp.columns = cols
    df_temp.drop(columns = 'temp', inplace = True) 

def processing_date():
    df_temp.rename(columns = {df_temp.columns[0]:'datetime'}, inplace = True)
    df_temp[['date', 'time']] = df_temp['datetime'].astype(str).str.split(' ', expand = True)
    df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format = "%d.%m.%Y %H:%M")
    df_temp['date'] = pd.to_datetime(df_temp['date'], format = "%d.%m.%Y")
    df_temp['month'] = df_temp.date.dt.month
    df_temp['year'] = df_temp.date.dt.year
    
    start, end = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][-2:]
    df_temp['start'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = start.day))
    df_temp['end'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = end.day))
    return (start, end)

def get_snowiness(df_group):
    df_winter = df_group.reset_index().pivot(columns = 'month', index = ['start', 'end'], values = 'RRR')
    months = df_winter.columns.to_list()
    df_winter['sum_RRR_fact'] = df_winter.sum(1)
    statistic = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][months]
    df_winter['percent'] = (df_winter.sum_RRR_fact/statistic.sum()*100)-100

    df_w = df_winter.copy()
    st = statistic.copy()
    st[start.month] = round(31-start.day, -1)//10/3*st[start.month]
    st[end.month] = round(end.day, -1)//10/3*st[end.month]
    
    df_w[start.month] = round(31-start.day, -1)//10/3*df_w[start.month]
    df_w[end.month] = round(end.day, -1)//10/3*df_w[end.month]
    df_w['decade_sum_RRR_fact'] = df_w[months].sum(1)
    df_w['decade_percent'] = (df_w.decade_sum_RRR_fact/st.sum()*100)-100

    df_winter['decade_start'] = df_w[start.month]
    df_winter['decade_end'] = df_w[end.month]
    df_winter['start_month'] = start.month
    df_winter['end_month'] = end.month
    df_winter['sum_RRR'] = statistic.sum()
    df_winter['decade_sum_RRR'] = st.sum()
    df_winter['decade_sum_RRR_fact'] = df_w['decade_sum_RRR_fact']
    df_winter['decade_percent'] = df_w['decade_percent']
    
    return df_winter

errors_station = df_snow.loc[(df_snow.id_station.isna()) | (df_snow.start.isna())].id_station.to_list()
df_snow['id_station'] = df_snow.id_station.astype(str)
dfs = []
datalist = df_snow.loc[(~df_snow.id_station.isna()) & ~(df_snow.start.isna())].id_station
for s in notebook.tqdm(datalist, total = len(datalist)):
    names = ['.01.01.2017.31.12.2020.1.0.0.ru.utf8.00000000','.26.02.2020.31.12.2020.1.0.0.ru.utf8.00000000', '.30.10.2017.31.12.2020.1.0.0.ru.utf8.00000000']
    for n in names:
        try:
#             print('data/'+str(s)+n+'.csv')
            df_temp = pd.read_csv('data/'+s+n+'.csv', skiprows = 6, sep = ';')
            break
        except:
            continue
        else:
            break
    else:
        print(s, 'not found file')
        errors_station.append(s)

    if s not in errors_station:
         
        changing_columns()

        df_temp['RRR'] = df_temp['RRR'].replace('Осадков нет', 0)
        df_temp['RRR'] = df_temp['RRR'].replace('Следы осадков', 0).astype(float)

        start, end = processing_date()
#         if s == '30317':
#             end -= datetime.timedelta(days=30)

        dates = df_temp.loc[(df_temp.WW.str.contains('снег')) | (df_temp.WW.str.contains('снеж')) |
                            (df_temp.W1.astype(str).str.contains('снег')) | (df_temp.W1.astype(str).str.contains('снеж')) |
                            (df_temp.W2.astype(str).str.contains('снег')) | (df_temp.W2.astype(str).str.contains('снеж')) |
                            (df_temp["E'"].astype(str).str.contains('снег')) | (df_temp["E'"].astype(str).str.contains('снеж')) |
                            (df_temp["sss"].astype(str).str.contains('снег')) | (df_temp["sss"].astype(str).str.contains('снеж')) |
                            (~df_temp.sss.isna())][['date']].drop_duplicates()['date']
        try:
            df_group = df_temp.loc[(df_temp.date.apply(lambda x: (x>=pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = 1)) \
                                    and (x<=pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, 
                                                         month = end.month, 
                                                         day = (datetime.date(x.year+1, end.month+1, 1) - datetime.timedelta(days=1)).day) ))) 
                    & (df_temp.date.isin(dates))\
                    & (df_temp.date >= pd.Timestamp(df_temp.date.dt.year.min(), start.month, 1)) \
                    & (df_temp.date <= pd.Timestamp(df_temp.date.dt.year.max(), end.month, (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day))
                   ].sort_values('date')\
                    .groupby(['start', 'end', 'month']).sum()[['RRR']]
            
            df_winter = get_snowiness(df_group)
            df_winter['id_station'] = s

            dfs.append(df_winter.reset_index())
        except Exception as e:
            print(s, e, 'len(dates): ', len(dates), 'min_date:', dates.min(), 'max_date:', dates.max())
            errors_station.append(s)
    

29590 index 10 is out of bounds for axis 0 with size 0 len(dates):  125 2020-02-27 00:00:00 2020-12-30 00:00:00
30507 index 10 is out of bounds for axis 0 with size 0 len(dates):  114 2020-02-27 00:00:00 2020-12-30 00:00:00
30514 index 10 is out of bounds for axis 0 with size 0 len(dates):  112 2020-02-27 00:00:00 2020-12-30 00:00:00
nan not found file
30791 index 11 is out of bounds for axis 0 with size 0 len(dates):  0 NaT NaT
UIBB not found file
nan not found file



In [421]:
errors_station

['29794',
 '29892',
 '30439',
 '30537',
 '30632',
 '30815',
 '29590',
 '30507',
 '30514',
 'nan',
 '30791',
 'UIBB',
 'nan']

In [422]:
df = pd.concat(dfs, ignore_index = True)

In [423]:
len(df.id_station.drop_duplicates())

56

In [424]:
df = df.merge(df_snow[['id_station', 'station']], how = 'left', on = 'id_station')

In [428]:
df = df[[
    'id_station',
    'station',
    'start',
    'end',
    10,
    11,
    12,
    1,
    2,
    3,
    4,
    'start_month',
    'end_month',
    'decade_start',
    'decade_end',
    'sum_RRR_fact',
    'sum_RRR',
    'percent',
    'decade_sum_RRR_fact',
    'decade_sum_RRR',
    'decade_percent',
]]

In [429]:
df

,id_station,station,start,end,10,11,12,1,2,3,...,start_month,end_month,decade_start,decade_end,sum_RRR_fact,sum_RRR,percent,decade_sum_RRR_fact,decade_sum_RRR,decade_percent
0,24817,Ербогачен,2017-10-11,2018-04-21,16.6,43.0,26.5,18.0,18.6,14.1,...,10,4,11.066667,9.133333,150.5,197.0,-23.604061,140.400000,168.333333,-16.594059
1,24817,Ербогачен,2018-10-11,2019-04-21,23.7,23.2,28.4,25.5,21.0,31.1,...,10,4,15.800000,15.333333,175.9,197.0,-10.710660,160.333333,168.333333,-4.752475
2,24817,Ербогачен,2019-10-11,2020-04-21,43.8,18.9,20.6,21.6,21.3,23.9,...,10,4,29.200000,1.000000,151.6,197.0,-23.045685,136.500000,168.333333,-18.910891
3,24918,Преображенка,2017-10-13,2018-04-15,20.9,29.1,33.3,24.0,14.6,13.1,...,10,4,13.933333,3.266667,139.9,218.0,-35.825688,131.300000,186.666667,-29.660714
4,24918,Преображенка,2018-10-13,2019-04-15,19.9,16.3,27.7,22.3,19.0,15.9,...,10,4,13.266667,19.466667,150.3,218.0,-31.055046,133.933333,186.666667,-28.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,30812,Култук,2018-10-27,2019-03-30,8.5,6.6,0.8,3.1,0.8,15.8,...,10,3,0.000000,15.800000,35.6,160.0,-77.750000,27.100000,122.000000,-77.786885
158,30812,Култук,2019-10-27,2020-03-30,1.0,4.6,1.9,1.3,6.6,6.4,...,10,3,0.000000,6.400000,21.8,160.0,-86.375000,20.800000,122.000000,-82.950820
159,30818,Байкальск,2017-10-19,2018-04-10,0.7,74.7,32.3,21.0,17.7,21.2,...,10,4,0.233333,12.700000,205.7,171.0,20.292398,179.833333,120.333333,49.445983
160,30818,Байкальск,2018-10-19,2019-04-10,32.8,21.0,25.6,13.4,3.9,28.4,...,10,4,10.933333,5.000000,140.1,171.0,-18.070175,108.233333,120.333333,-10.055402


In [433]:
df.loc[df.decade_percent>=25]

,id_station,station,start,end,10,11,12,1,2,3,...,start_month,end_month,decade_start,decade_end,sum_RRR_fact,sum_RRR,percent,decade_sum_RRR_fact,decade_sum_RRR,decade_percent
126,30703,Инга,2017-11-04,2018-03-23,NaN,30.2,45.5,18.1,22.7,22.0,...,11,3,30.200000,14.666667,138.5,95.0,45.789474,131.166667,83.666667,56.772908
141,30715,Ангарск,2017-11-01,2018-03-23,NaN,14.3,48.6,12.7,19.1,13.6,...,11,3,14.300000,9.066667,108.3,86.0,25.930233,103.766667,78.333333,32.468085
159,30818,Байкальск,2017-10-19,2018-04-10,0.7,74.7,32.3,21.0,17.7,21.2,...,10,4,0.233333,12.700000,205.7,171.0,20.292398,179.833333,120.333333,49.445983


In [431]:
df.to_csv('data/snowiness.csv', index = False)

# Проверки

In [411]:
df_temp = pd.read_csv('data/30317.01.01.2017.31.12.2020.1.0.0.ru.utf8.00000000.csv', skiprows = 6, sep = ';')

In [412]:
cols = list(df_temp.columns) 
cols.append('temp')
df_temp.reset_index(inplace = True)
df_temp.columns = cols
df_temp.drop(columns = 'temp', inplace = True) 

In [413]:
df_temp.rename(columns = {df_temp.columns[0]:'datetime'}, inplace = True)

In [414]:
df_temp[['date', 'time']] = df_temp['datetime'].astype(str).str.split(' ', expand = True)
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format = "%d.%m.%Y %H:%M")
df_temp['date'] = pd.to_datetime(df_temp['date'], format = "%d.%m.%Y")

In [415]:
s = '30317'

In [416]:
df_temp['month'] = df_temp.date.dt.month
df_temp['year'] = df_temp.date.dt.year

In [417]:
df_temp['RRR'] = df_temp['RRR'].replace('Осадков нет', 0)
df_temp['RRR'] = df_temp['RRR'].replace('Следы осадков', 0).astype(float)
start, end = df_snow.loc[df_snow.id_station=='30317'].set_index('id_station').iloc[0][-2:]
# end = pd.Timestamp(year= end.year, month = end.month - 1, day = 31)
df_temp['start'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = start.day))
df_temp['end'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = end.day))


In [405]:
dates = df_temp.loc[(df_temp.WW.str.contains('снег')) | (df_temp.WW.str.contains('снеж')) |
            (df_temp.W1.astype(str).str.contains('снег')) | (df_temp.W1.astype(str).str.contains('снеж')) |
            (df_temp.W2.astype(str).str.contains('снег')) | (df_temp.W2.astype(str).str.contains('снеж')) |
            (df_temp["E'"].astype(str).str.contains('снег')) | (df_temp["E'"].astype(str).str.contains('снеж')) |
            (df_temp["sss"].astype(str).str.contains('снег')) | (df_temp["sss"].astype(str).str.contains('снеж')) |
            (~df_temp.sss.isna())][['date']].drop_duplicates()['date']

df_group = df_temp.loc[(df_temp.date.apply(lambda x: (x>=pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = 1)) \
                                    and (x<=pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = 1) ))) 
                    & (df_temp.date.isin(dates))\
                    & (df_temp.date >= pd.Timestamp(df_temp.date.dt.year.min(), start.month, 1)) \
                    & (df_temp.date <= pd.Timestamp(df_temp.date.dt.year.max(), end.month, (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day))
                   ].sort_values('date')\
                    .groupby(['start', 'end', 'month']).sum()[['RRR']]
            
df_winter = df_group.reset_index().pivot(columns = 'month', index = ['start', 'end'], values = 'RRR')
months = df_winter.columns.to_list()
df_winter['sum_RRR_fact'] = df_winter.sum(1)
statistic = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][months].sum()
df_winter['percent'] = (df_winter.sum_RRR_fact/statistic*100)-100

df_w = df_winter.copy()
df_w[start.month] = round(31-start.day, -1)//10/3*df_w[start.month]
df_w[end.month] = round(end.day, -1)//10/3*df_w[end.month]
df_w['decade_sum_RRR'] = df_w[months].sum(1)
df_w['decade_percent'] = (df_w.decade_sum_RRR/statistic*100)-100

df_winter['decade_start'] = df_w[start.month]
df_winter['decade_end'] = df_w[end.month]
df_winter['start_month'] = start.month
df_winter['end_month'] = end.month
df_winter['sum_RRR'] = statistic
df_winter['decade_sum_RRR'] = df_w['decade_sum_RRR']
df_winter['decade_percent'] = df_w['decade_percent']
df_winter['id_station'] = s


KeyError: 3

In [418]:
df_temp.loc[(df_temp.date.apply(lambda x: (x>=pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = 1)) \
                                    and (x<=pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, 
                                                         month = end.month, 
                                                         day = (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day) ))) 
                    & (df_temp.date.isin(dates))\
                    & (df_temp.date >= pd.Timestamp(df_temp.date.dt.year.min(), start.month, 1)) \
                    & (df_temp.date <= pd.Timestamp(df_temp.date.dt.year.max(), end.month, (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day))
                   ].sort_values('date')\
                    .groupby(['start', 'end', 'month']).sum()[['RRR']]

RRR
start      end        month      
2017-10-17 2018-04-02 1       4.7
                      2       1.3
                      3       4.4
                      4      11.1
                      10     18.6
                      11     28.4
                      12     25.0
2018-10-17 2019-04-02 1       1.6
                      2       0.9
                      3       5.0
                      4      13.0
                      10      2.1
                      11     59.3
                      12      9.4
2019-10-17 2020-04-02 1       6.1
                      2       4.2
                      3      14.5
                      4       0.0
                      10     11.2
                      11     37.0
                      12      8.2